# AI重构唐诗：诗人风格与时代脉络分析

**基于深度学习文本向量化的《全唐诗》风格聚类研究**

---

## 课程简介

本实验将带你体验如何使用人工智能技术分析《全唐诗》，探索诗人之间的风格传承关系。

通过本实验，你将：

1. 了解**文本向量化**技术如何将诗歌转化为数学表示
2. 学会使用AI计算诗人之间的**风格相似度**
3. 掌握**聚类分析**和**降维可视化**的基本原理
4. 体验**语义检索**：用现代白话文搜索古诗
5. 通过**词云**直观感受不同时代的诗歌特征
6. 探索基于历史文献的**诗人社交网络**

---

## 数据来源与处理说明

| 数据集 | 来源 | 用途 |
|--------|------|------|
| chinese-poetry | GitHub开源项目 | 诗歌文本 |
| CBDB | 哈佛大学/北京大学/中研院 | 诗人生卒年等元数据 |

**繁简转换说明**：
- 原始数据为繁体中文，繁简转换存在「一对多」问题（如：後/后、發/髮）
- 为保证效果，本实验采用混合方案：繁体原文用于AI向量化，简体版本用于界面显示

---

## 目录

- [第零部分: 环境配置与准备](#part0)
- [第一部分: 数据下载与加载](#part1)
- [第二部分: 数据探索与预处理](#part2)
- [第三部分: AI文本向量化](#part3)
- [第四部分: 诗人风格相似度分析](#part4)
- [第五部分: 风格聚类与可视化](#part5)
- [第六部分: 交互式风格探索器](#part6)
- [第七部分: 诗歌语义检索](#part7)
- [第八部分: 词云可视化](#part8)
- [第九部分: 诗人社交网络](#part9)
- [附录: 常见问题与拓展](#appendix)

---
<a id='part0'></a>
# 第零部分: 环境配置与准备

In [1]:
#@title ## 0.1 检查运行环境 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，检查GPU环境。
#@markdown ---

import sys
import os

print("=" * 60)
print(" 环境检测")
print("=" * 60)
print(f" Python 版本: {sys.version.split()[0]}")

IN_COLAB = 'google.colab' in sys.modules
print(f" 运行环境: {'Google Colab' if IN_COLAB else '本地环境'}")

try:
    import subprocess
    result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'],
                          capture_output=True, text=True)
    if result.returncode == 0:
        print(f" GPU 可用: {result.stdout.strip()}")
    else:
        print(" 未检测到 GPU，请在菜单中选择 运行时 -> 更改运行时类型 -> GPU")
except:
    print(" 无法检测 GPU")

print("=" * 60)

 环境检测
 Python 版本: 3.12.12
 运行环境: Google Colab
 GPU 可用: NVIDIA A100-SXM4-40GB, 40960 MiB


In [2]:
#@title ## 0.2 安装必要的库 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，安装本实验需要的Python库。
#@markdown
#@markdown ---

print(" 正在安装依赖...")
print("-" * 40)

import subprocess
import sys
import warnings
warnings.filterwarnings('ignore')

packages = [
    ('sentence-transformers', 'sentence_transformers'),
    ('umap-learn', 'umap'),
    ('opencc-python-reimplemented', 'opencc'),
    ('plotly', 'plotly'),
    ('gradio', 'gradio'),
    ('scikit-learn', 'sklearn'),
    ('wordcloud', 'wordcloud'),      # 词云
    ('jieba', 'jieba'),              # 中文分词
    ('networkx', 'networkx'),        # 网络图
    ('pyvis', 'pyvis'),              # 网络可视化
]

for i, (pkg_name, import_name) in enumerate(packages, 1):
    print(f"[{i}/{len(packages)}] 检查 {pkg_name}...")
    try:
        __import__(import_name)
        print(f"    已安装")
    except ImportError:
        print(f"    正在安装...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pkg_name])
        print(f"    完成")

print("-" * 40)
print(" 所有依赖安装完成!")

 正在安装依赖...
----------------------------------------
[1/10] 检查 sentence-transformers...
    已安装
[2/10] 检查 umap-learn...
    已安装
[3/10] 检查 opencc-python-reimplemented...
    正在安装...
    完成
[4/10] 检查 plotly...
    已安装
[5/10] 检查 gradio...
    已安装
[6/10] 检查 scikit-learn...
    已安装
[7/10] 检查 wordcloud...
    已安装
[8/10] 检查 jieba...
    已安装
[9/10] 检查 networkx...
    已安装
[10/10] 检查 pyvis...
    正在安装...
    完成
----------------------------------------
 所有依赖安装完成!


In [4]:
#@title ## 0.3 导入库并配置环境 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，导入所有需要的库。
#@markdown ---

import json
import glob
import os
import sqlite3
import numpy as np
import pandas as pd
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

import opencc
converter = opencc.OpenCC('t2s')

from sentence_transformers import SentenceTransformer

import umap
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import gradio as gr

np.random.seed(42)

print("=" * 60)
print(" 库导入完成")
print("=" * 60)
print(f" NumPy: {np.__version__}")
print(f" Pandas: {pd.__version__}")
print(" 繁简转换: opencc (t2s)")
print("=" * 60)

 库导入完成
 NumPy: 2.0.2
 Pandas: 2.2.2
 繁简转换: opencc (t2s)


---
<a id='part1'></a>
# 第一部分: 数据下载与加载

In [5]:
#@title ## 1.1 下载全唐诗数据集 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，从GitHub下载chinese-poetry数据集。
#@markdown ---

DATA_DIR = 'chinese-poetry'

if not os.path.exists(DATA_DIR):
    print(" 正在下载全唐诗数据集...")
    print("-" * 40)
    !git clone --depth 1 https://github.com/chinese-poetry/chinese-poetry.git
    print("-" * 40)
    print(" 下载完成!")
else:
    print(" 数据集已存在，跳过下载")

tang_files = glob.glob(f"{DATA_DIR}/全唐诗/poet.tang.*.json")
print(f" 找到 {len(tang_files)} 个唐诗数据文件")

 正在下载全唐诗数据集...
----------------------------------------
Cloning into 'chinese-poetry'...
remote: Enumerating objects: 2277, done.
remote: Counting objects: 100% (2277/2277), done.
remote: Compressing objects: 100% (1166/1166), done.
remote: Total 2277 (delta 1143), reused 1340 (delta 1104), pack-reused 0 (from 0)
Receiving objects: 100% (2277/2277), 94.60 MiB | 11.99 MiB/s, done.
Resolving deltas: 100% (1143/1143), done.
Updating files: 100% (2285/2285), done.
----------------------------------------
 下载完成!
 找到 58 个唐诗数据文件


In [6]:
#@title ## 1.2 下载CBDB人物数据库 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，下载哈佛大学CBDB数据库（约200MB）。
#@markdown
#@markdown CBDB: 哈佛大学、北京大学、台湾中研院联合开发，65万+历史人物
#@markdown
#@markdown ---

CBDB_FILE = 'cbdb_sqlite.db'

if not os.path.exists(CBDB_FILE):
    print(" 正在下载CBDB数据库...")
    print("-" * 40)
    !wget -q --show-progress -O cbdb.7z "https://github.com/cbdb-project/cbdb_sqlite/raw/master/latest.7z"
    !apt-get install -y -qq p7zip-full
    !7z x -y cbdb.7z -o./cbdb_temp/ > /dev/null
    db_files = glob.glob('./cbdb_temp/*.db')
    if db_files:
        os.rename(db_files[0], CBDB_FILE)
    !rm -rf cbdb.7z cbdb_temp/
    print("-" * 40)
    print(" CBDB下载完成!")
else:
    print(" CBDB数据库已存在，跳过下载")

file_size = os.path.getsize(CBDB_FILE) / (1024*1024)
print(f" 数据库大小: {file_size:.1f} MB")

 正在下载CBDB数据库...
----------------------------------------
cbdb.7z             100%[===================>]  78.03M  17.2MB/s    in 4.5s    
----------------------------------------
 CBDB下载完成!
 数据库大小: 506.7 MB


In [7]:
#@title ## 1.3 加载唐诗数据（繁简双版本） { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，加载全唐诗数据。
#@markdown
#@markdown 处理策略：
#@markdown - 保留**繁体原文**用于AI分析（语义更准确）
#@markdown - 生成**简体版本**用于界面显示（方便阅读）
#@markdown ---

def load_tang_poems(data_dir='chinese-poetry/全唐诗'):
    poems_data = []
    json_files = sorted(glob.glob(f"{data_dir}/poet.tang.*.json"))

    print(f" 正在加载 {len(json_files)} 个文件...")
    print("-" * 40)

    for idx, file_path in enumerate(json_files):
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        for poem in data:
            full_text = ''.join(poem.get('paragraphs', []))
            author = poem.get('author', '佚名')
            title = poem.get('title', '無題')

            poems_data.append({
                'author': author,
                'author_simp': converter.convert(author),
                'title': title,
                'title_simp': converter.convert(title),
                'content': full_text,
                'content_simp': converter.convert(full_text),
                'char_count': len(full_text)
            })

        if (idx + 1) % 10 == 0 or idx == len(json_files) - 1:
            print(f" 已加载: {idx + 1}/{len(json_files)} 文件, 共 {len(poems_data)} 首诗")

    print("-" * 40)
    return pd.DataFrame(poems_data)

df_poems = load_tang_poems()

print("\n" + "=" * 60)
print(" 数据加载完成")
print("=" * 60)
print(f" 总诗歌数: {len(df_poems)}")
print(f" 诗人数量: {df_poems['author'].nunique()}")

print("\n 数据示例：")
print("-" * 40)
s = df_poems.iloc[0]
print(f" 作者: {s['author_simp']} (繁:{s['author']})")
print(f" 标题: {s['title_simp']}")
print(f" 内容: {s['content_simp'][:30]}...")
print("=" * 60)

 正在加载 58 个文件...
----------------------------------------
 已加载: 10/58 文件, 共 10000 首诗
 已加载: 20/58 文件, 共 20009 首诗
 已加载: 30/58 文件, 共 30012 首诗
 已加载: 40/58 文件, 共 40018 首诗
 已加载: 50/58 文件, 共 50012 首诗
 已加载: 58/58 文件, 共 57607 首诗
----------------------------------------

 数据加载完成
 总诗歌数: 57607
 诗人数量: 3663

 数据示例：
----------------------------------------
 作者: 太宗皇帝 (繁:太宗皇帝)
 标题: 帝京篇十首 一
 内容: 秦川雄帝宅，函谷壮皇居。绮殿千寻起，离宫百雉余。连甍遥接汉，...


---
<a id='part2'></a>
# 第二部分: 数据探索与预处理

In [8]:
#@title ## 2.1 诗人作品数量统计 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，统计各诗人的作品数量。
#@markdown ---

poet_counts = df_poems['author'].value_counts()
poet_name_map = df_poems.drop_duplicates('author')[['author', 'author_simp']].set_index('author')['author_simp'].to_dict()

print("=" * 60)
print(" 诗人作品数量统计")
print("=" * 60)

print("\n 作品数最多的20位诗人：")
print("-" * 40)
for i, (poet, count) in enumerate(poet_counts.head(20).items(), 1):
    print(f" {i:2d}. {poet_name_map.get(poet, poet):10s} : {count:5d} 首")

print("\n" + "-" * 40)
print(f" 作品数 >= 50首的诗人: {len(poet_counts[poet_counts >= 50])} 位")
print("=" * 60)

 诗人作品数量统计

 作品数最多的20位诗人：
----------------------------------------
  1. 白居易        :  3009 首
  2. 杜甫         :  1489 首
  3. 李白         :  1207 首
  4. 元稹         :   910 首
  5. 刘禹锡        :   886 首
  6. 不详         :   886 首
  7. 齐己         :   826 首
  8. 贯休         :   743 首
  9. 易静         :   721 首
 10. 无名氏        :   684 首
 11. 陆龟蒙        :   635 首
 12. 李商隐        :   616 首
 13. 韦应物        :   586 首
 14. 王建         :   580 首
 15. 张祜         :   565 首
 16. 杜牧         :   544 首
 17. 钱起         :   541 首
 18. 许浑         :   537 首
 19. 姚合         :   534 首
 20. 孟郊         :   532 首

----------------------------------------
 作品数 >= 50首的诗人: 186 位


In [ ]:
#@title ## 2.2 可视化作品数量分布 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，绘制诗人作品数量分布图。
#@markdown ---

top_30 = poet_counts.head(30)
top_30_simp = pd.Series({poet_name_map.get(p, p): c for p, c in top_30.items()})

fig = make_subplots(rows=1, cols=2, subplot_titles=('作品数量最多的30位诗人', '诗人作品数量分布'))

fig.add_trace(go.Bar(x=top_30_simp.values, y=top_30_simp.index, orientation='h', marker_color='steelblue'), row=1, col=1)
fig.add_trace(go.Histogram(x=poet_counts.values, nbinsx=50, marker_color='coral'), row=1, col=2)
fig.add_vline(x=50, line_dash="dash", line_color="red", row=1, col=2, annotation_text="阈值=50", annotation_position="top right")

fig.update_layout(height=600, width=1200, showlegend=False)
fig.update_yaxes(categoryorder='total ascending', row=1, col=1)
fig.update_yaxes(type='log', title='诗人数量(对数)', row=1, col=2)
fig.update_xaxes(title='作品数', row=1, col=1)
fig.update_xaxes(title='作品数', row=1, col=2)

fig.show()

In [9]:
#@title ## 2.3 筛选核心诗人 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，筛选作品数>=50首的核心诗人。
#@markdown ---

MIN_POEMS = 50
EXCLUDE_AUTHORS = ['佚名', '無名氏', '不詳']

valid_poets = poet_counts[(poet_counts >= MIN_POEMS) & (~poet_counts.index.isin(EXCLUDE_AUTHORS))].index.tolist()

print("=" * 60)
print(" 核心诗人筛选结果")
print("=" * 60)
print(f" 筛选条件: 作品数 >= {MIN_POEMS}首")
print(f" 筛选后诗人数: {len(valid_poets)} 位")

df_selected = df_poems[df_poems['author'].isin(valid_poets)].copy()
print(f" 筛选后诗歌数: {len(df_selected)} 首")
print("=" * 60)

 核心诗人筛选结果
 筛选条件: 作品数 >= 50首
 筛选后诗人数: 183 位
 筛选后诗歌数: 43175 首


In [10]:
#@title ## 2.4 从CBDB提取诗人元数据 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，从CBDB提取唐代诗人的生卒年信息。
#@markdown
#@markdown CBDB数据来源：哈佛大学、北京大学、台湾中研院联合编纂
#@markdown ---

print(" 正在连接CBDB数据库...")
print("-" * 40)

conn = sqlite3.connect(CBDB_FILE)
query = "SELECT c_personid, c_name_chn, c_birthyear, c_deathyear FROM BIOG_MAIN WHERE c_dy = 6"
df_cbdb_tang = pd.read_sql(query, conn)
print(f" CBDB唐代人物总数: {len(df_cbdb_tang)}")

matched_poets = []
unmatched_poets = []

for poet in valid_poets:
    match = df_cbdb_tang[df_cbdb_tang['c_name_chn'] == poet]

    if len(match) > 0:
        row = match.iloc[0]
        birth = row['c_birthyear'] if pd.notna(row['c_birthyear']) and row['c_birthyear'] > 0 else None
        death = row['c_deathyear'] if pd.notna(row['c_deathyear']) and row['c_deathyear'] > 0 else None

        active_year = None
        if birth and death:
            active_year = (birth + death) // 2
        elif birth:
            active_year = birth + 30
        elif death:
            active_year = death - 30

        if active_year:
            if active_year < 713: era = '初唐'
            elif active_year < 766: era = '盛唐'
            elif active_year < 836: era = '中唐'
            else: era = '晚唐'
        else:
            era = '未知'

        matched_poets.append({
            'poet': poet,
            'poet_simp': poet_name_map.get(poet, poet),
            'cbdb_id': int(row['c_personid']),
            'birth': int(birth) if birth else None,
            'death': int(death) if death else None,
            'era': era,
            'poem_count': poet_counts[poet]
        })
    else:
        unmatched_poets.append(poet)

conn.close()

df_poet_meta = pd.DataFrame(matched_poets)
for poet in unmatched_poets:
    df_poet_meta = pd.concat([df_poet_meta, pd.DataFrame([{
        'poet': poet, 'poet_simp': poet_name_map.get(poet, poet),
        'cbdb_id': None, 'birth': None, 'death': None,
        'era': '未知', 'poem_count': poet_counts[poet]
    }])], ignore_index=True)

print("-" * 40)
print("=" * 60)
print(" CBDB数据匹配完成")
print("=" * 60)
print(f" 成功匹配: {len(matched_poets)} 位 | 未匹配: {len(unmatched_poets)} 位")

era_counts = df_poet_meta['era'].value_counts()
print("\n 各时代诗人分布：")
for era in ['初唐', '盛唐', '中唐', '晚唐', '未知']:
    if era in era_counts.index:
        print(f"   {era}: {era_counts[era]} 位")
print("=" * 60)

 正在连接CBDB数据库...
----------------------------------------
 CBDB唐代人物总数: 53953
----------------------------------------
 CBDB数据匹配完成
 成功匹配: 146 位 | 未匹配: 37 位

 各时代诗人分布：
   初唐: 16 位
   盛唐: 24 位
   中唐: 36 位
   晚唐: 22 位
   未知: 85 位


---
<a id='part3'></a>
# 第三部分: AI文本向量化

使用BGE中文模型将诗歌转换为向量表示（使用繁体原文，语义更准确）。

In [11]:
#@title ## 3.1 加载中文向量化模型 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，加载BGE中文文本向量化模型。
#@markdown
#@markdown 模型: `BAAI/bge-base-zh-v1.5` (北京智源人工智能研究院)
#@markdown
#@markdown ---

print(" 正在加载中文向量化模型...")
print("-" * 40)

MODEL_NAME = 'BAAI/bge-base-zh-v1.5'
model = SentenceTransformer(MODEL_NAME)

print("-" * 40)
print("=" * 60)
print(" 模型加载完成")
print("=" * 60)
print(f" 模型: {MODEL_NAME}")
print(f" 向量维度: {model.get_sentence_embedding_dimension()}")
print(f" 输入文本: 繁体原文")
print("=" * 60)

 正在加载中文向量化模型...
----------------------------------------


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/409M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

----------------------------------------
 模型加载完成
 模型: BAAI/bge-base-zh-v1.5
 向量维度: 768
 输入文本: 繁体原文


In [12]:
#@title ## 3.2 诗歌向量化 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，将诗歌转换为向量（使用繁体原文）。
#@markdown
#@markdown ---

print(" 正在对诗歌进行向量化（使用繁体原文）...")
print("-" * 40)

poems_content = df_selected['content'].tolist()
print(f" 待处理诗歌数: {len(poems_content)}")

BATCH_SIZE = 128
poem_embeddings = model.encode(poems_content, batch_size=BATCH_SIZE, show_progress_bar=True, convert_to_numpy=True)

df_selected['embedding'] = list(poem_embeddings)

print("-" * 40)
print(f" 向量矩阵形状: {poem_embeddings.shape}")

 正在对诗歌进行向量化（使用繁体原文）...
----------------------------------------
 待处理诗歌数: 43175


Batches:   0%|          | 0/338 [00:00<?, ?it/s]

----------------------------------------
 向量矩阵形状: (43175, 768)


In [13]:
#@title ## 3.3 计算诗人风格向量 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，计算每位诗人的平均风格向量。
#@markdown ---

print(" 正在计算诗人风格向量...")
print("-" * 40)

poet_vectors = {}
for poet in valid_poets:
    poet_poems = df_selected[df_selected['author'] == poet]
    if len(poet_poems) > 0:
        vectors = np.array(poet_poems['embedding'].tolist())
        poet_vectors[poet] = vectors.mean(axis=0)

poet_names = list(poet_vectors.keys())
poet_vector_matrix = np.array([poet_vectors[p] for p in poet_names])

print(f" 诗人数量: {len(poet_names)}")
print(f" 向量矩阵形状: {poet_vector_matrix.shape}")

 正在计算诗人风格向量...
----------------------------------------
 诗人数量: 183
 向量矩阵形状: (183, 768)


---
<a id='part4'></a>
# 第四部分: 诗人风格相似度分析

In [16]:
#@title ## 4.1 计算风格相似度矩阵 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，计算所有诗人之间的余弦相似度。
#@markdown ---

print(" 正在计算诗人风格相似度矩阵...")

similarity_matrix = cosine_similarity(poet_vector_matrix)
df_similarity = pd.DataFrame(similarity_matrix, index=poet_names, columns=poet_names)

print(f" 矩阵形状: {similarity_matrix.shape}")
print(f" 相似度范围: [{similarity_matrix.min():.3f}, {similarity_matrix.max():.3f}]")

 正在计算诗人风格相似度矩阵...
 矩阵形状: (183, 183)
 相似度范围: [0.762, 1.000]


In [17]:
#@title ## 4.2 查找最相似的诗人 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，为代表性诗人找出风格最相似的诗人。
#@markdown ---

def find_most_similar(poet_name, df_sim, top_n=5):
    if poet_name not in df_sim.index:
        return []
    similarities = df_sim[poet_name].drop(poet_name)
    return list(zip(similarities.nlargest(top_n).index, similarities.nlargest(top_n).values))

def get_poet_era(poet_name):
    row = df_poet_meta[df_poet_meta['poet'] == poet_name]
    return row.iloc[0]['era'] if len(row) > 0 else '未知'

def get_poet_simp(poet_name):
    return poet_name_map.get(poet_name, poet_name)

showcase_poets = ['李白', '杜甫', '王維', '白居易', '李商隱']

print("=" * 60)
print(" 诗人风格相似度分析")
print("=" * 60)

for poet in showcase_poets:
    if poet in df_similarity.index:
        similar = find_most_similar(poet, df_similarity, top_n=5)
        print(f"\n {get_poet_simp(poet)}（{get_poet_era(poet)}）风格最相似的诗人：")
        print("-" * 40)
        for rank, (name, score) in enumerate(similar, 1):
            print(f"   {rank}. {get_poet_simp(name)}（{get_poet_era(name)}）: {score:.3f}")

print("\n" + "=" * 60)

 诗人风格相似度分析

 李白（盛唐）风格最相似的诗人：
----------------------------------------
   1. 卢照邻（初唐）: 0.991
   2. 鲍溶（未知）: 0.990
   3. 李颀（未知）: 0.990
   4. 王维（盛唐）: 0.989
   5. 宋之问（初唐）: 0.989

 杜甫（盛唐）风格最相似的诗人：
----------------------------------------
   1. 岑参（盛唐）: 0.993
   2. 罗隐（未知）: 0.992
   3. 王维（盛唐）: 0.992
   4. 李颀（未知）: 0.991
   5. 黄滔（晚唐）: 0.991

 王维（盛唐）风格最相似的诗人：
----------------------------------------
   1. 卢纶（未知）: 0.996
   2. 刘禹锡（中唐）: 0.995
   3. 欧阳詹（中唐）: 0.994
   4. 岑参（盛唐）: 0.993
   5. 李端（盛唐）: 0.993

 白居易（中唐）风格最相似的诗人：
----------------------------------------
   1. 元稹（中唐）: 0.996
   2. 姚合（中唐）: 0.992
   3. 杜牧（中唐）: 0.992
   4. 唐彦谦（晚唐）: 0.992
   5. 刘禹锡（中唐）: 0.991

 李商隐（中唐）风格最相似的诗人：
----------------------------------------
   1. 杜牧（中唐）: 0.996
   2. 唐彦谦（晚唐）: 0.996
   3. 刘禹锡（中唐）: 0.995
   4. 吴融（晚唐）: 0.994
   5. 韩偓（晚唐）: 0.994



In [18]:
#@title ## 4.3 绘制相似度热力图 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，绘制诗人风格相似度热力图（前40位）。
#@markdown
#@markdown 热力图展示诗人两两之间的风格相似度。
#@markdown
#@markdown ### 如何读懂热力图
#@markdown
#@markdown | 颜色 | 相似度 | 含义 |
#@markdown |------|--------|------|
#@markdown | 深红色 | 接近1.0 | 风格高度相似 |
#@markdown | 浅色 | 0.7-0.9 | 有一定相似性 |
#@markdown | 深蓝色 | 低于0.7 | 风格差异较大 |
#@markdown | 对角线 | 1.0 | 诗人与自己的相似度 |
#@markdown
#@markdown ### 观察要点
#@markdown
#@markdown **寻找"红色方块"**：
#@markdown - 热力图中出现的红色区块，代表一群风格相近的诗人
#@markdown - 这些诗人是否属于同一时代？同一流派？
#@markdown
#@markdown **寻找"冷门关联"**：
#@markdown - 两个看似无关的诗人之间出现红色，说明AI发现了隐藏的风格关联
#@markdown - 这种关联能否用文学史知识解释？
#@markdown
#@markdown **寻找"异类"**：
#@markdown - 某一行/列整体偏蓝，说明该诗人风格独特，与大多数人不同
#@markdown - 谁是唐诗中的"风格孤岛"？
#@markdown
#@markdown ### 图表交互
#@markdown
#@markdown - **悬停**：查看任意两位诗人的具体相似度数值
#@markdown - **缩放**：放大观察局部细节
#@markdown
#@markdown ### 思考题
#@markdown 1. 热力图是否呈现出"分块"现象？这些块对应什么？
#@markdown 2. 作品数量最多的诗人（白居易、杜甫），相似度分布有何特点？
#@markdown 3. 为什么大部分相似度都在0.9以上？这对分析有什么影响？
#@markdown 4. 如果要选10位诗人代表唐诗的多样性，你会如何利用这张图来选择？
#@markdown ---
TOP_N = 40
top_poets = poet_names[:TOP_N]
sim_subset = df_similarity.loc[top_poets, top_poets]
top_poets_simp = [get_poet_simp(p) for p in top_poets]

fig = go.Figure(data=go.Heatmap(
    z=sim_subset.values, x=top_poets_simp, y=top_poets_simp,
    colorscale='RdBu_r', zmid=0.5,
    hovertemplate='%{y} vs %{x}<br>相似度: %{z:.3f}<extra></extra>'
))

fig.update_layout(title='诗人风格相似度热力图', width=900, height=800, xaxis={'tickangle': 45})
fig.show()

---
<a id='part5'></a>
# 第五部分: 风格聚类与可视化

In [19]:
#@title ## 5.1 UMAP降维与K-Means聚类 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，进行降维和聚类分析。
#@markdown ---

print(" 正在进行UMAP降维...")
reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, metric='cosine', random_state=42)
poet_2d = reducer.fit_transform(poet_vector_matrix)

print(" 正在进行K-Means聚类...")
N_CLUSTERS = 8
kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(poet_vector_matrix)

df_cluster = pd.DataFrame({
    'poet': poet_names,
    'poet_simp': [get_poet_simp(p) for p in poet_names],
    'x': poet_2d[:, 0], 'y': poet_2d[:, 1],
    'cluster': cluster_labels,
    'era': [get_poet_era(p) for p in poet_names],
    'poem_count': [poet_counts.get(p, 0) for p in poet_names]
})

print(f" 降维: {poet_vector_matrix.shape[1]}D -> 2D")
print(f" 聚类数: {N_CLUSTERS}")

 正在进行UMAP降维...
 正在进行K-Means聚类...
 降维: 768D -> 2D
 聚类数: 8


In [ ]:
#@title ## 5.2 可视化诗人风格分布 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，绘制诗人风格散点图。
#@markdown
#@markdown 将生成两张图：按AI聚类着色 和 按历史时代着色。
#@markdown
#@markdown ### 观察要点
#@markdown
#@markdown **图1：AI聚类图**
#@markdown - 相同颜色的诗人被AI判定为风格相近
#@markdown - 观察：同一聚类中的诗人，你认为他们有什么共同点？
#@markdown - 思考：AI聚类与传统的"流派"划分（边塞派、田园派等）是否吻合？
#@markdown
#@markdown **图2：时代分布图**
#@markdown - 颜色代表历史时代：蓝=初唐 | 橙=盛唐 | 绿=中唐 | 红=晚唐
#@markdown - 观察：同一时代的诗人是否聚集在一起？
#@markdown - 思考：风格演变是否呈现出时间上的"轨迹"？
#@markdown
#@markdown ### 图表交互
#@markdown
#@markdown - **悬停**：查看诗人姓名、时代、作品数
#@markdown - **缩放**：滚轮或双指缩放，观察局部细节
#@markdown - **拖动**：移动视图，探索边缘区域的诗人
#@markdown
#@markdown ### 思考题
#@markdown 1. 图中位置靠近的诗人，风格一定相似吗？为什么？
#@markdown 2. 有没有"跨时代"的风格群落？（如初唐和晚唐诗人混在一起）
#@markdown 3. 哪些诗人处于图的边缘？这意味着他们的风格有何特点？
#@markdown 4. 如果把宋词诗人加入，你觉得他们会出现在图的什么位置？
#@markdown ---

# 按聚类着色
fig1 = px.scatter(df_cluster, x='x', y='y', color='cluster', hover_name='poet_simp',
                  hover_data=['era', 'poem_count'], title='唐代诗人风格聚类图（按AI聚类）',
                  color_continuous_scale='Viridis', width=900, height=700)

for _, row in df_cluster.nlargest(20, 'poem_count').iterrows():
    fig1.add_annotation(x=row['x'], y=row['y'], text=row['poet_simp'], showarrow=False, font=dict(size=10))

fig1.show()

# 按时代着色
era_colors = {'初唐': '#1f77b4', '盛唐': '#ff7f0e', '中唐': '#2ca02c', '晚唐': '#d62728', '未知': '#7f7f7f'}
fig2 = px.scatter(df_cluster, x='x', y='y', color='era', hover_name='poet_simp',
                  title='唐代诗人风格分布图（按历史时代）',
                  category_orders={'era': ['初唐', '盛唐', '中唐', '晚唐', '未知']},
                  color_discrete_map=era_colors, width=900, height=700)
fig2.show()

print(" 蓝色:初唐 | 橙色:盛唐 | 绿色:中唐 | 红色:晚唐")

 蓝色:初唐 | 橙色:盛唐 | 绿色:中唐 | 红色:晚唐


---
<a id='part6'></a>
# 第六部分: 交互式诗人风格探索

In [20]:
#@title ## 6.1 诗人风格探索器 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，启动交互式诗人风格探索界面。
#@markdown
#@markdown 选择任意诗人，查看其风格特征和最相似的诗人。
#@markdown
#@markdown ###  探索建议
#@markdown
#@markdown **验证文学史常识**：
#@markdown - 查看"李白"，最相似的是否有"杜甫"？（诗仙vs诗圣）
#@markdown - 查看"白居易"，是否与"元稹"高度相似？（元白并称）
#@markdown - 查看"李商隐"，是否与"杜牧"相似？（小李杜）
#@markdown
#@markdown **发现意外关联**：
#@markdown - 跨时代的风格传承：初唐诗人与盛唐诗人有何关联？
#@markdown - 同时代的风格差异：同为盛唐，李白与王维有多相似？
#@markdown - 冷门诗人：那些你不熟悉的诗人，与谁风格接近？
#@markdown
#@markdown ###  观察要点
#@markdown
#@markdown - **相似度数值**：0.99+表示极高相似，0.95以下表示有明显差异
#@markdown - **时代分布**：与某诗人相似的诗人，是否集中在同一时代？
#@markdown - **作品数量**：作品多的诗人，风格特征是否更稳定？
#@markdown
#@markdown ###  思考题
#@markdown 1. AI计算的"风格相似"与文学史的"流派归属"是否一致？
#@markdown 2. 为什么有些公认的"对手"（如李白vs杜甫）AI却认为相似？
#@markdown 3. 风格相似一定意味着存在影响关系吗？还可能有哪些原因？
#@markdown 4. 如果你是古代诗人，想学习某种风格，AI会推荐你读谁的诗？
#@markdown ---

simp_to_trad = {v: k for k, v in poet_name_map.items() if k in poet_names}

def get_poet_info(poet_name):
    row = df_poet_meta[df_poet_meta['poet'] == poet_name]
    if len(row) > 0:
        r = row.iloc[0]
        birth = int(r['birth']) if pd.notna(r['birth']) else '?'
        death = int(r['death']) if pd.notna(r['death']) else '?'
        return r['era'], f"{birth}-{death}"
    return '未知', '?-?'

def explore_poet_style(poet_simp, top_n=10):
    poet_trad = simp_to_trad.get(poet_simp, poet_simp)
    if poet_trad not in df_similarity.index:
        return f"未找到诗人: {poet_simp}"

    era, years = get_poet_info(poet_trad)

    output = ["=" * 50, f" 诗人: {poet_simp}", "=" * 50]
    output.append(f" 时代: {era} | 生卒: {years}（来源：哈佛CBDB）")
    output.append(f" 作品数: {poet_counts.get(poet_trad, 0)}")

    output.append("\n" + "-" * 50)
    output.append(f" 风格最相似的 {top_n} 位诗人：")
    output.append("-" * 50)

    for rank, (name, score) in enumerate(find_most_similar(poet_trad, df_similarity, top_n), 1):
        other_era, other_years = get_poet_info(name)
        output.append(f" {rank:2d}. {get_poet_simp(name):8s} | {score:.3f} | {other_era} | {other_years}")

    output.append("\n" + "-" * 50)
    output.append(" 代表作品（简体显示）：")
    output.append("-" * 50)

    for _, poem in df_selected[df_selected['author'] == poet_trad].head(5).iterrows():
        title = poem['title_simp'][:12] + '...' if len(poem['title_simp']) > 12 else poem['title_simp']
        content = poem['content_simp'][:25] + '...' if len(poem['content_simp']) > 25 else poem['content_simp']
        output.append(f" [{title}] {content}")

    return '\n'.join(output)

poet_names_simp = sorted([get_poet_simp(p) for p in poet_names])

demo = gr.Interface(
    fn=explore_poet_style,
    inputs=[gr.Dropdown(choices=poet_names_simp, value='李白', label='选择诗人'),
            gr.Slider(minimum=5, maximum=20, value=10, step=1, label='相似诗人数量')],
    outputs=gr.Textbox(label='分析结果', lines=25),
    title='唐诗诗人风格探索器',
    description='选择诗人查看风格特征（元数据来源：哈佛CBDB）',
    allow_flagging='never'
)

demo.launch(share=True, debug=False, quiet=True)

* Running on public URL: https://fe3523c0357247e0ac.gradio.live


---
<a id='part7'></a>
# 第七部分: 诗歌语义检索

In [21]:
#@title ## 7.1 诗歌语义检索 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，启动诗歌语义检索功能。
#@markdown
#@markdown 输入任意文字（关键词、句子、诗句），AI会找出语义最相似的诗歌。
#@markdown
#@markdown ###  检索建议
#@markdown
#@markdown **用现代白话文检索古诗**：
#@markdown - "思念远方的亲人" → 找出表达思乡之情的诗
#@markdown - "战争的残酷" → 找出边塞战争相关的诗
#@markdown - "隐居山林的生活" → 找出田园隐逸诗
#@markdown
#@markdown **用意象关键词检索**：
#@markdown - "月亮 思念" → 月下怀人的诗
#@markdown - "落花 流水" → 伤春惜时的诗
#@markdown - "长安 繁华" → 描写都城的诗
#@markdown
#@markdown **用情感词检索**：
#@markdown - "孤独寂寞" → 表达孤寂情绪的诗
#@markdown - "豪情壮志" → 抒发壮志的诗
#@markdown - "离别伤感" → 送别诗
#@markdown
#@markdown ###  观察要点
#@markdown
#@markdown - 检索结果中，哪些诗人出现频率最高？这说明什么？
#@markdown - 同一主题下，不同诗人的表达方式有何差异？
#@markdown - AI理解的"语义相似"与你的直觉是否一致？
#@markdown
#@markdown ###  思考题
#@markdown 1. 为什么用"想家"能搜到古诗？AI是如何理解古今语义关联的？
#@markdown 2. 检索结果的相似度都很高（0.9+），这意味着什么？
#@markdown 3. 这种语义检索技术可以应用在哪些场景？（诗歌推荐、文学研究、教育...）
#@markdown ---

def search_poems(query_text, top_n=10):
    """语义检索：根据输入文字找出最相似的诗歌"""
    if not query_text.strip():
        return "请输入检索内容"

    # 将查询文本向量化
    query_embedding = model.encode(query_text, convert_to_numpy=True)

    # 计算与所有诗歌的相似度
    similarities = cosine_similarity([query_embedding], poem_embeddings)[0]

    # 获取最相似的诗歌索引
    top_indices = similarities.argsort()[-top_n:][::-1]

    output = []
    output.append("=" * 60)
    output.append(f" 检索词: 「{query_text}」")
    output.append("=" * 60)
    output.append(f" 找到最相似的 {top_n} 首诗：\n")

    for rank, idx in enumerate(top_indices, 1):
        row = df_selected.iloc[idx]
        sim_score = similarities[idx]

        output.append(f"【{rank}】相似度: {sim_score:.4f}")
        output.append(f"  《{row['title_simp']}》 —— {row['author_simp']}")

        # 显示诗歌内容（简体，限制长度）
        content = row['content_simp']
        if len(content) > 60:
            content = content[:60] + "..."
        output.append(f"  {content}")
        output.append("")

    return "\n".join(output)

# 创建Gradio界面
demo_search = gr.Interface(
    fn=search_poems,
    inputs=[
        gr.Textbox(label="输入检索内容", placeholder="例如：思念故乡、月亮、边塞战争、春天的花..."),
        gr.Slider(minimum=5, maximum=20, value=10, step=1, label="返回结果数量")
    ],
    outputs=gr.Textbox(label="检索结果", lines=30),
    title=" 唐诗语义检索",
    description="输入任意文字，AI会找出语义最相似的唐诗。支持现代白话文检索古诗！",
    examples=[
        ["思念远方的亲人", 10],
        ["边塞战争", 10],
        ["春天的美景", 10],
        ["月光下的思念", 10],
        ["归隐山林", 10],
    ],
    allow_flagging='never'
)

demo_search.launch(share=True, debug=False, quiet=True)

* Running on public URL: https://4addd2baee01ffe5be.gradio.live


---
<a id='part8'></a>
# 第八部分: 词云可视化

In [22]:
#@title ## 8.1 词云可视化 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，生成诗人/时代的词云图。
#@markdown
#@markdown ###  探索建议
#@markdown
#@markdown **按时代对比**：依次生成初唐→盛唐→中唐→晚唐的词云，观察：
#@markdown - 哪些意象词（如"明月""春风"）贯穿始终？
#@markdown - 哪些情感词（如"惆怅""可怜"）在特定时期出现？
#@markdown - 否定词（"不知""不见""不得"）的频率如何变化？
#@markdown
#@markdown **按诗人对比**：比较同时代不同诗人的词云，思考：
#@markdown - 李白 vs 杜甫：谁更浪漫？谁更现实？
#@markdown - 王维 vs 岑参：山水田园 vs 边塞风光？
#@markdown - 白居易 vs 李商隐：通俗直白 vs 朦胧绮丽？
#@markdown
#@markdown ###  思考题
#@markdown 1. 为什么"夕阳"只在晚唐高频出现？这与时代背景有何关联？
#@markdown 2. AI发现的高频词与你对唐诗的印象是否一致？有什么意外发现？
#@markdown 3. 词频分析的局限性是什么？它能否完全代表诗人风格？
#@markdown ---

import jieba
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from collections import Counter
import os

# 查找可用的中文字体
def find_chinese_font():
    font_paths = [
        '/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc',
        '/usr/share/fonts/truetype/wqy/wqy-microhei.ttc',
        '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc',
    ]
    for path in font_paths:
        if os.path.exists(path):
            return path
    os.system('apt-get install -y -qq fonts-wqy-zenhei')
    return '/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc'

FONT_PATH = find_chinese_font()

# 停用词
STOPWORDS = set([
    # 古诗常见虚词
    '之', '乎', '者', '也', '兮', '而', '何', '其', '於', '为', '以', '不',
    '无', '有', '是', '如', '与', '及', '则', '乃', '若', '或', '且', '然',
    # 数字
    '一', '二', '三', '四', '五', '六', '七', '八', '九', '十', '百', '千', '万',
    # 常见词
    '人', '中', '上', '下', '来', '去', '日', '月', '年', '时', '里', '处',
    # 校勘注释用语
    '一作', '项校', '改作', '六卷', '伯三', '三七', '六五', '三五', '四六',
    '全诗', '原作', '原注', '此诗', '此句', '一本', '或作', '又作', '本作',
    '校记', '卷数', '注云', '案语'
])

def generate_wordcloud(poet_or_era, mode='poet'):
    """生成词云"""
    try:
        if mode == 'poet':
            poet_trad = simp_to_trad.get(poet_or_era, poet_or_era)
            poems = df_selected[df_selected['author'] == poet_trad]['content_simp'].tolist()
            title = f"{poet_or_era} 诗歌词云"
        else:
            poems = df_cluster[df_cluster['era'] == poet_or_era].merge(
                df_selected, left_on='poet', right_on='author'
            )['content_simp'].tolist()
            title = f"{poet_or_era} 诗歌词云"

        if len(poems) == 0:
            return None, "未找到相关诗歌"

        all_text = ' '.join(poems)
        words = jieba.lcut(all_text)
        words = [w for w in words if len(w) >= 2 and w not in STOPWORDS and w.isalpha()]
        word_freq = Counter(words)

        if len(word_freq) == 0:
            return None, "没有足够的词汇生成词云"

        wc = WordCloud(
            font_path=FONT_PATH,
            width=800,
            height=600,
            background_color='white',
            max_words=150,
            colormap='viridis',
            random_state=42
        )
        wc.generate_from_frequencies(word_freq)

        # 绑图
        fig, ax = plt.subplots(figsize=(12, 8))
        ax.imshow(wc, interpolation='bilinear')
        ax.axis('off')
        font_prop = fm.FontProperties(fname=FONT_PATH, size=16)
        ax.set_title(title, fontproperties=font_prop, fontweight='bold')
        plt.tight_layout()

        # 词频统计
        top_words = word_freq.most_common(20)
        stats = f"诗歌数量: {len(poems)}\n\n高频词 Top 20:\n"
        stats += "\n".join([f"  {w}: {c}" for w, c in top_words])

        return fig, stats

    except Exception as e:
        return None, f"生成词云出错: {str(e)}"

# Gradio界面
with gr.Blocks(title="词云可视化") as demo_wordcloud:
    gr.Markdown("# 唐诗词云可视化")
    gr.Markdown("选择诗人或时代，生成其诗歌的词云图")

    with gr.Row():
        with gr.Column():
            mode = gr.Radio(["poet", "era"], value="poet", label="选择模式",
                           info="poet=按诗人，era=按时代")
            name_input = gr.Dropdown(choices=poet_names_simp, value="李白", label="选择诗人")
            era_input = gr.Dropdown(choices=['初唐', '盛唐', '中唐', '晚唐'], value="盛唐", label="选择时代")
            btn = gr.Button("生成词云", variant="primary")

        with gr.Column():
            output_img = gr.Plot(label="词云图")
            output_stats = gr.Textbox(label="词频统计", lines=15)

    def update_wordcloud(mode, poet, era):
        name = poet if mode == 'poet' else era
        return generate_wordcloud(name, mode)

    btn.click(
        fn=update_wordcloud,
        inputs=[mode, name_input, era_input],
        outputs=[output_img, output_stats]
    )

demo_wordcloud.launch(share=True, debug=False, quiet=True)

* Running on public URL: https://bd316a73e3478f74c1.gradio.live


---
<a id='part9'></a>
# 第九部分: 诗人社交网络

In [23]:
#@title ## 9.1 从CBDB提取诗人社会关系 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，从CBDB数据库提取诗人之间的社会关系。
#@markdown

#@markdown CBDB记录的关系类型包括：亲属、师生、同僚、友人等。
#@markdown ---


import networkx as nx

print(" 正在从CBDB提取社会关系...")
print("-" * 40)

conn = sqlite3.connect(CBDB_FILE)

# 获取我们诗人的CBDB ID
poet_ids = df_poet_meta[df_poet_meta['cbdb_id'].notna()]['cbdb_id'].astype(int).tolist()
poet_id_to_name = dict(zip(
    df_poet_meta[df_poet_meta['cbdb_id'].notna()]['cbdb_id'].astype(int),
    df_poet_meta[df_poet_meta['cbdb_id'].notna()]['poet_simp']
))

# 查询社会关系表（表名是 ASSOC_DATA）
query = """
SELECT
    a.c_personid,
    a.c_assoc_id,
    a.c_assoc_code,
    c.c_assoc_desc_chn
FROM ASSOC_DATA a
LEFT JOIN ASSOC_CODES c ON a.c_assoc_code = c.c_assoc_code
WHERE a.c_personid IN ({})
  AND a.c_assoc_id IN ({})
""".format(','.join(map(str, poet_ids)), ','.join(map(str, poet_ids)))

try:
    df_relations = pd.read_sql(query, conn)
    print(f" 找到诗人之间的关系: {len(df_relations)} 条")
except Exception as e:
    print(f" 查询出错: {e}")
    df_relations = pd.DataFrame()

conn.close()

# 构建网络图
G = nx.Graph()

# 添加节点（所有有CBDB ID的诗人）
for pid, name in poet_id_to_name.items():
    era = df_poet_meta[df_poet_meta['cbdb_id'] == pid]['era'].values
    era = era[0] if len(era) > 0 else '未知'
    G.add_node(name, era=era)

# 添加边（社会关系）
if len(df_relations) > 0:
    for _, row in df_relations.iterrows():
        person1 = poet_id_to_name.get(int(row['c_personid']))
        person2 = poet_id_to_name.get(int(row['c_assoc_id']))
        rel_type = row.get('c_assoc_desc_chn', '关联')

        if person1 and person2 and person1 != person2:
            if G.has_edge(person1, person2):
                G[person1][person2]['relation'] += f", {rel_type}"
            else:
                G.add_edge(person1, person2, relation=rel_type if rel_type else '关联')

print("-" * 40)
print(f" 网络节点数: {G.number_of_nodes()}")
print(f" 网络边数: {G.number_of_edges()}")
print("=" * 60)

 正在从CBDB提取社会关系...
----------------------------------------
 找到诗人之间的关系: 982 条
----------------------------------------
 网络节点数: 146
 网络边数: 400


In [24]:
#@title ## 9.2 可视化诗人社交网络 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，绘制诗人社交网络图。
#@markdown
#@markdown 颜色代表时代，连线代表CBDB记录的社会关系。
#@markdown ---

import plotly.graph_objects as go

# 只保留关系数>=5的诗人
MIN_RELATIONS = 5
nodes_with_relations = [n for n in G.nodes() if G.degree(n) >= MIN_RELATIONS]
G_filtered = G.subgraph(nodes_with_relations)

print(f" 筛选条件: 关系数 ≥ {MIN_RELATIONS}")
print(f" 筛选后诗人数: {len(nodes_with_relations)} 位")
print(f" 关系连线数: {G_filtered.number_of_edges()}")
print("-" * 40)

# 时代颜色映射
era_colors = {
    '初唐': '#1f77b4',
    '盛唐': '#ff7f0e',
    '中唐': '#2ca02c',
    '晚唐': '#d62728',
    '未知': '#7f7f7f'
}

# 使用spring布局
pos = nx.spring_layout(G_filtered, k=3, iterations=100, seed=42)

# 创建边
edge_x = []
edge_y = []
for edge in G_filtered.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#cccccc'),
    hoverinfo='none',
    mode='lines',
    showlegend=False
)

# 按时代分组创建节点
node_traces = []
MIN_DEGREE_FOR_LABEL = 10  # 关系数>=10才显示名字

for era, color in era_colors.items():
    era_nodes = [n for n in G_filtered.nodes() if G_filtered.nodes[n].get('era', '未知') == era]
    if not era_nodes:
        continue

    node_x = []
    node_y = []
    node_text = []
    node_size = []
    node_labels = []

    for node in era_nodes:
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

        degree = G_filtered.degree(node)
        node_text.append(f"{node}<br>时代: {era}<br>关系数: {degree}")
        node_size.append(12 + degree * 1.5)

        if degree >= MIN_DEGREE_FOR_LABEL:
            node_labels.append(node)
        else:
            node_labels.append('')

    node_traces.append(go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        hoverinfo='text',
        text=node_labels,
        textposition="top center",
        textfont=dict(size=10, color='black'),
        hovertext=node_text,
        marker=dict(
            color=color,
            size=node_size,
            line=dict(width=0.5, color='white')
        ),
        name=era
    ))

# 绑制图形
fig = go.Figure(
    data=[edge_trace] + node_traces,
    layout=go.Layout(
        title=f'唐代诗人社交网络（基于CBDB历史关系数据）<br><sub>仅显示关系数≥{MIN_RELATIONS}的诗人</sub>',
        showlegend=True,
        hovermode='closest',
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        width=1000,
        height=800,
        legend=dict(title='时代', x=1.02, y=1),
        plot_bgcolor='white'
    )
)

fig.show()

print("\n 图表说明：")
print(" - 节点大小：社交关系越多，节点越大")
print(f" - 仅显示关系数≥{MIN_DEGREE_FOR_LABEL}的诗人名字")
print(" - 悬停节点可查看诗人详细信息")
print(" - 连线代表CBDB记录的历史社会关系（亲属、师生、同僚、友人等）")

 筛选条件: 关系数 ≥ 5
 筛选后诗人数: 67 位
 关系连线数: 321
----------------------------------------



 图表说明：
 - 节点大小：社交关系越多，节点越大
 - 仅显示关系数≥10的诗人名字
 - 悬停节点可查看诗人详细信息
 - 连线代表CBDB记录的历史社会关系（亲属、师生、同僚、友人等）


In [25]:
#@title ## 9.3 分析社交网络特征 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，分析诗人社交网络的特征。
#@markdown
#@markdown ###  网络指标解读
#@markdown
#@markdown | 指标 | 含义 | 解读 |
#@markdown |------|------|------|
#@markdown | **社交关系数** | 诗人直接关联的人数 | 数值越大，社交越活跃 |
#@markdown | **网络密度** | 实际关系数/最大可能关系数 | 越接近1，诗人圈子越紧密 |
#@markdown | **平均路径长度** | 任意两人之间的平均"跳数" | 越小说明圈子越小 |
#@markdown | **聚类系数** | 朋友之间也互相认识的程度 | 越高说明"抱团"越明显 |
#@markdown
#@markdown ###  观察要点
#@markdown
#@markdown **时代分布**：观察社交活跃的诗人主要集中在哪个时代？为什么？
#@markdown - 中唐诗人为何占据主导？（提示：文人集团、诗歌唱和）
#@markdown - 初唐、盛唐诗人的社交网络有何特点？
#@markdown
#@markdown **核心人物**：谁是唐代诗坛的"社交中心"？
#@markdown - 白居易、韩愈为何关系最多？
#@markdown - 李白、杜甫的社交模式有何不同？
#@markdown
#@markdown ###  思考题
#@markdown 1. 社交关系多的诗人，文学影响力一定更大吗？
#@markdown 2. 为什么会有"独立社交圈"？这反映了什么历史现象？
#@markdown 3. CBDB记录的关系包括亲属、师生、同僚、友人等，这些关系对诗歌创作有何影响？
#@markdown 4. 对比AI风格相似度和历史社交网络，有社交关系的诗人风格一定相似吗？
#@markdown ---

print("=" * 60)
print(" 诗人社交网络分析")
print("=" * 60)

# 度数最高的诗人（社交关系最多）
degree_dict = dict(G.degree())
top_connected = sorted(degree_dict.items(), key=lambda x: x[1], reverse=True)[:15]

print("\n 社交关系最多的诗人 Top 15：")
print("-" * 40)
for rank, (poet, degree) in enumerate(top_connected, 1):
    era = G.nodes[poet].get('era', '未知')
    print(f" {rank:2d}. {poet}（{era}）: {degree} 个关系")

# 查找社交圈（连通分量）
components = list(nx.connected_components(G))
print(f"\n 独立社交圈数量: {len(components)}")

# 最大社交圈
largest_cc = max(components, key=len)
print(f" 最大社交圈人数: {len(largest_cc)}")

# 网络密度
density = nx.density(G)
print(f"\n 网络密度: {density:.4f}")

# 平均路径长度（只计算最大连通分量）
if len(largest_cc) > 1:
    subgraph = G.subgraph(largest_cc)
    try:
        avg_path = nx.average_shortest_path_length(subgraph)
        print(f" 平均路径长度: {avg_path:.2f}")
    except:
        print(" 平均路径长度: 无法计算")

# 聚类系数
clustering = nx.average_clustering(G)
print(f" 平均聚类系数: {clustering:.4f}")

print("\n" + "=" * 60)

 诗人社交网络分析

 社交关系最多的诗人 Top 15：
----------------------------------------
  1. 白居易（中唐）: 27 个关系
  2. 杜甫（盛唐）: 25 个关系
  3. 刘禹锡（中唐）: 23 个关系
  4. 贾岛（中唐）: 23 个关系
  5. 韩愈（中唐）: 22 个关系
  6. 李白（盛唐）: 21 个关系
  7. 姚合（中唐）: 18 个关系
  8. 张籍（中唐）: 17 个关系
  9. 张祜（中唐）: 15 个关系
 10. 方干（晚唐）: 15 个关系
 11. 杨巨源（中唐）: 15 个关系
 12. 元稹（中唐）: 14 个关系
 13. 王建（中唐）: 14 个关系
 14. 杜牧（中唐）: 14 个关系
 15. 孟郊（中唐）: 13 个关系

 独立社交圈数量: 40
 最大社交圈人数: 107

 网络密度: 0.0378
 平均路径长度: 2.69
 平均聚类系数: 0.2400




---
<a id='appendix'></a>
# 附录

---

## A. 常见问题 (FAQ)

**Q1: 为什么采用繁简混合方案？**

A: 繁简转换存在「一对多」问题，会导致语义信息损失：
- 「後」（时间）、「后」（皇后）→ 都变成「后」
- 「發」（发出）、「髮」（头发）→ 都变成「发」
- 「乾」（天）、「幹」（做）→ 都变成「干」

因此我们用繁体原文进行AI向量化（保证语义准确），简体版本用于界面显示（方便阅读）。

**Q2: 为什么有些诗人显示「未知」时代？**

A: CBDB数据库虽然收录了该诗人，但缺少生卒年记录，无法自动判断时代。这在历史数据中很常见。

**Q3: 诗人相似度都很高（0.98+），是否正常？**

A: 正常。因为所有样本都是唐诗，基础语义相似度本来就很高。关键是看**相对排名**，而非绝对数值。

**Q4: AI聚类结果与传统流派划分不一致？**

A: AI基于文本语义特征聚类，与传统文学史的流派划分（如边塞派、田园派）角度不同。这正是AI分析的价值——可能发现传统方法忽略的风格关联。

**Q5: 时代划分的依据是什么？**

A: 采用通行的四分法：
- 初唐 (618-712)：高祖→玄宗前
- 盛唐 (713-765)：开元盛世→安史之乱
- 中唐 (766-835)：代宗→文宗
- 晚唐 (836-907)：武宗→唐亡

注：学界分期略有差异，部分诗人（如李商隐）的归属存在争议。

**Q6: 能否分析宋诗、宋词？**

A: 可以！chinese-poetry数据集包含26万首宋诗和2.1万首宋词，只需修改数据加载路径即可扩展分析。

**Q7: 为什么词云中出现"一作""项校"等奇怪的词？**

A: 这些是古籍的校勘注释用语，混入了诗歌正文。我们已在停用词表中过滤掉大部分，但可能仍有遗漏。如发现类似问题，可自行扩展“词云可视化代码单元格”中停用词列表。

**Q8: 为什么有些诗歌内容出现□符号？**

A: □代表原始文献中的缺字。古籍流传过程中可能出现损坏、模糊等情况，数字化时无法识别的字符用□占位。这恰恰说明数据的学术严谨性。

**Q9: 热力图中为什么有明显的"蓝色十字线"？**

A: 这些是风格异类诗人，如王梵志（白话诗）、易静（道教诗）、贯休/齐己（僧人诗）等。他们的诗歌语言和题材与主流文人诗差异较大，AI准确识别出了这种风格边界。

**Q10: 社交网络中的关系数据可靠吗？**

A: CBDB的社会关系数据来自哈佛大学、北京大学等学术机构对历史文献的考证，具有较高的学术可信度。关系类型包括亲属、师生、同僚、友人等。

---

## B. 拓展资源

| 资源 | 链接 | 说明 |
|------|------|------|
| chinese-poetry | https://github.com/chinese-poetry/chinese-poetry | 本实验诗歌数据来源 |
| CBDB | https://projects.iq.harvard.edu/cbdb | 哈佛中国历代人物传记数据库 |
| 全唐诗库 | https://ctext.org/quantangshi/zhs | 中国哲学书电子化计划 |
| BGE模型 | https://huggingface.co/BAAI/bge-base-zh-v1.5 | 北京智源中文向量模型 |
| Sentence-BERT | https://www.sbert.net/ | 文本向量化框架 |
| UMAP文档 | https://umap-learn.readthedocs.io/ | 降维算法官方文档 |
| jieba分词 | https://github.com/fxsjy/jieba | 中文分词工具 |
| WordCloud | https://github.com/amueller/word_cloud | 词云生成库 |
| NetworkX | https://networkx.org/ | 网络分析库 |

---

## C. 进阶学习方向

如果你对数字人文和AI文本分析感兴趣，可以进一步探索：

1. **主题建模**：使用LDA或BERTopic提取诗歌主题，分析不同时代的题材演变
2. **情感分析**：分析诗歌的情感倾向，比较不同诗人的情感特征
3. **地理可视化**：结合CBDB籍贯数据，绘制唐诗人地理分布图
4. **跨朝代对比**：将分析扩展到宋诗、宋词，观察风格演变
5. **诗歌生成**：基于GPT模型生成特定风格的诗歌
6. **深度社会网络**：扩展CBDB关系分析，研究诗人的师承谱系和文学集团
7. **时序分析**：结合诗人创作年代，分析个人风格的演变轨迹
8. **跨语言检索**：用英文检索中国古诗，探索跨语言语义理解

---

## D. 术语表

| 术语 | 解释 |
|------|------|
| 文本向量化 | 将文本转换为数值向量，便于计算机处理 |
| 余弦相似度 | 衡量两个向量夹角的相似度指标，范围[-1,1] |
| UMAP | 统一流形近似与投影，一种非线性降维算法 |
| K-Means | 经典的聚类算法，将数据分为K个簇 |
| Embedding | 嵌入向量，文本的高维数值表示 |
| BGE | 智源开源的中文文本向量模型 |
| CBDB | 中国历代人物传记资料库 |
| 繁简转换 | 繁体中文与简体中文之间的转换 |
| OpenCC | 开源的中文繁简转换工具 |
| Gradio | 快速构建机器学习交互界面的Python库 |
| jieba | 优秀的中文分词工具 |
| 词云 | 以词频控制字体大小的可视化方式 |
| 停用词 | 分析时需要过滤掉的无意义高频词 |
| 社交网络 | 表示人物之间社会关系的图结构 |
| 网络密度 | 实际连接数占最大可能连接数的比例 |
| 聚类系数 | 衡量网络中节点聚集程度的指标 |
| 语义检索 | 基于语义相似度而非关键词匹配的搜索方式 |

---

## E. 注意事项

1. **数据局限性**：chinese-poetry数据来自网络整理，可能存在录入错误或缺字（□）
2. **元数据匹配**：CBDB与诗歌数据的姓名匹配基于字符串精确匹配，同名异人或异名同人可能导致误差
3. **模型局限性**：BGE模型基于现代中文语料训练，对古典诗歌的语义理解可能不够精准
4. **解释谨慎**：AI发现的风格关联需要结合文学史知识进行解读，避免过度诠释
5. **计算资源**：完整运行本实验需要GPU支持，建议使用Colab的免费GPU
6. **链接有效期**：Gradio生成的公开链接有效期1周，重新运行会生成新链接
7. **词云质量**：分词效果依赖jieba词典，古诗中的生僻词可能切分不准确
8. **社交网络完整性**：CBDB记录的关系可能不完整，未记录不代表没有关系

---

*本教学Notebook基于开源数据和模型开发，仅供教学与研究使用。*

*如有问题或建议，欢迎反馈！*